In [1]:
import numpy as np
import bipartitepandas as bpd
import pytwoway as tw
from pytwoway import constraints as cons
from matplotlib import pyplot as plt


nl = 3 # Number of worker types
nk = 3 # Number of firm types
blm_params = tw.blm_params({
    'nl': nl, 'nk': nk,
    'a1_mu': 0, 'a1_sig': 1.5, 'a2_mu': 0, 'a2_sig': 1.5,
    's1_low': 0.5, 's1_high': 1.5, 's2_low': 0.5, 's2_high': 1.5, 'verbose': 0
})
cluster_params = bpd.cluster_params({
    'measures': bpd.measures.CDFs(),
    'grouping': bpd.grouping.KMeans(n_clusters=nk),
    'is_sorted': True,
    'copy': False
})
clean_params = bpd.clean_params({
    'drop_returns': 'returners',
    'copy': False
})

/home/jovyan/.cache/pypoetry/virtualenvs/pytwoway-8EXZSVYp-py3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


real_values_a = []
real_values_psi = []
real_values_cov = []
estimated_a = []
estimated_psi = []
estimated_cov = []
monotonic_mover = []
monotonic_stayer = []

values = np.arange(0,3,0.05)

In [3]:
sim_params = bpd.sim_params({
    'nl': nl, 'nk': nk,
    'alpha_sig': 0.1, 'psi_sig': 1.0, 'w_sig': 0.6,
    'c_sort': 0, 'c_netw': 0, 'c_sig': 1
})

sim_data = bpd.SimBipartite(sim_params).simulate()

# Convert into BipartitePandas DataFrame
sim_data = bpd.BipartiteDataFrame(sim_data)
# Clean and collapse
sim_data = sim_data.clean(clean_params).collapse(is_sorted=True, copy=False)
# Cluster
sim_data = sim_data.cluster(cluster_params)
# Convert to event study format
sim_data = sim_data.to_eventstudy(is_sorted=True, copy=False)

sim_data_observed = sim_data.loc[:, ['i', 'j1', 'j2', 'y1', 'y2', 't11', 't12', 't21', 't22', 'g1', 'g2', 'w1', 'w2', 'm']]
sim_data_unobserved = sim_data.loc[:, ['alpha1', 'alpha2', 'k1', 'k2', 'l1', 'l2', 'psi1', 'psi2']]

movers = sim_data_observed.get_worker_m(is_sorted=True)
jdata = sim_data_observed.loc[movers, :]
sdata = sim_data_observed.loc[~movers, :]


# Initialize BLM estimator
blm_fit = tw.BLMEstimator(blm_params)
# Fit BLM estimator
blm_fit.fit(jdata=jdata, sdata=sdata, n_init=40, n_best=5, ncore=16)
# Store best model
blm_model = blm_fit.model

liks1 = blm_model.liks1

print('Log-likelihoods monotonic (movers):', np.min(np.diff(liks1)) >= 0)
monotonic_mover = monotonic_mover + [np.min(np.diff(liks1)) >= 0]
liks0 = blm_model.liks0
print('Log-likelihoods monotonic (stayers):', np.min(np.diff(liks0)) >= 0)
monotonic_stayer = monotonic_stayer + [np.min(np.diff(liks0)) >= 0]

# Initialize BLM variance decomposition estimator
blm_var = tw.BLMVarianceDecomposition(blm_model,blm_params)
# Fit BLM estimator
blm_var.fit(
    jdata=jdata, sdata=sdata,
    n_samples=20,
    Q_var=[
                tw.Q.VarAlpha(),
                tw.Q.VarPsi(),
                tw.Q.VarPsiPlusAlpha()
            ],
    ncore=16
)

for k, v in blm_var.res['var_decomp'].items():
    print(f'{k!r}:  {np.mean(v)}\n')

real_values_a = real_values_a + [np.var(sim_data_unobserved['alpha1'])]
real_values_psi = real_values_psi + [np.var(sim_data_unobserved['psi1'])]
real_values_cov = real_values_cov + [np.cov(sim_data_unobserved['psi1'], sim_data_unobserved['alpha1'], ddof=0)[0, 1]]
estimated_a = estimated_a + [np.mean(blm_var.res['var_decomp']['var(alpha)'])]
estimated_psi = estimated_psi + [np.mean(blm_var.res['var_decomp']['var(psi)'])]
estimated_cov = estimated_cov + [np.mean(blm_var.res['var_decomp']['cov(psi, alpha)'])]

checking required columns and datatypes
sorting rows
dropping NaN observations
generating 'm' column
keeping highest paying job for i-t (worker-year) duplicates (how='max')
dropping workers who leave a firm then return to it (how='returners')
making 'i' ids contiguous
making 'j' ids contiguous
computing largest connected set (how=None)
sorting columns
resetting index


  8%|▊         | 3/40 [00:08<01:40,  2.70s/it]

Maximum iterations reached for movers. It is recommended to increase `n_iters_movers` from its current value of 1000.


100%|██████████| 40/40 [00:10<00:00,  3.73it/s]

Log-likelihoods monotonic (movers): True
Log-likelihoods monotonic (stayers): True



100%|██████████| 20/20 [00:00<00:00, 80.10it/s]

'var(y)':  0.5268974090907124

'var(eps)':  0.14375285921709394

'var(alpha)':  0.10661254723604655

'var(psi + alpha)':  0.3831445498736182

'var(psi)':  0.2851741635299866

'cov(psi, alpha)':  -0.004321080446207527

